In [1]:
import fsspec
import cf_xarray #noqa
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.tri as tri
from kerchunk.netCDF3 import NetCDF3ToZarr
import ujson


fs_read = fsspec.filesystem('s3', anon=True, skip_instance_cache=True)
fs_write = fsspec.filesystem('')

In [2]:
vdatum_files = fs_read.glob('s3://noaa-nos-ofs-pds/OFS_Grid_Datum/*vdatums.nc')

vdatum_files = sorted(['s3://'+f for f in vdatum_files])
vdatum_files

['s3://noaa-nos-ofs-pds/OFS_Grid_Datum/cbofs_vdatums.nc',
 's3://noaa-nos-ofs-pds/OFS_Grid_Datum/ciofs_vdatums.nc',
 's3://noaa-nos-ofs-pds/OFS_Grid_Datum/creofs_vdatums.nc',
 's3://noaa-nos-ofs-pds/OFS_Grid_Datum/dbofs_vdatums.nc',
 's3://noaa-nos-ofs-pds/OFS_Grid_Datum/gomofs_vdatums.nc',
 's3://noaa-nos-ofs-pds/OFS_Grid_Datum/leofs_vdatums.nc',
 's3://noaa-nos-ofs-pds/OFS_Grid_Datum/lmhofs_vdatums.nc',
 's3://noaa-nos-ofs-pds/OFS_Grid_Datum/loofs_vdatums.nc',
 's3://noaa-nos-ofs-pds/OFS_Grid_Datum/lsofs_vdatums.nc',
 's3://noaa-nos-ofs-pds/OFS_Grid_Datum/mbmcf_vdatums.nc',
 's3://noaa-nos-ofs-pds/OFS_Grid_Datum/ngofs2_vdatums.nc',
 's3://noaa-nos-ofs-pds/OFS_Grid_Datum/nyofs_vdatums.nc',
 's3://noaa-nos-ofs-pds/OFS_Grid_Datum/sfbofs_vdatums.nc',
 's3://noaa-nos-ofs-pds/OFS_Grid_Datum/sjrofs_vdatums.nc',
 's3://noaa-nos-ofs-pds/OFS_Grid_Datum/tbmcf_vdatums.nc',
 's3://noaa-nos-ofs-pds/OFS_Grid_Datum/tbofs_vdatums.nc',
 's3://noaa-nos-ofs-pds/OFS_Grid_Datum/wcofs_vdatums.nc']

In [6]:
import ujson

def generate_json(f):
    outname = f'./vdatums/{f.split("/")[-1]}.zarr'

    chunks = NetCDF3ToZarr(f)
    refs = chunks.translate()
    with fs_write.open(outname, 'w') as ofile:
        ofile.write(ujson.dumps(refs))

    return outname

In [7]:
for f in vdatum_files:
    generate_json(f)

In [5]:
# open dataset as zarr object using fsspec reference file system and xarray
vdatum_fs = fsspec.filesystem(
    "reference",
    fo="./vdatums/ngofs2_vdatums.nc.zarr",
    remote_protocol="s3",
    remote_options={"anon": True},
)
vdatum_mapper = vdatum_fs.get_mapper("")

ds_vdatum = xr.open_dataset(
    vdatum_mapper, engine="zarr", chunks={}, backend_kwargs=dict(consolidated=False)
)
ds_vdatum

<xarray.Dataset>
Dimensions:      (N_node: 303714)
Dimensions without coordinates: N_node
Data variables:
    latitude     (N_node) float64 dask.array<chunksize=(303714,), meta=np.ndarray>
    longitude    (N_node) float64 dask.array<chunksize=(303714,), meta=np.ndarray>
    mhhwtomsl    (N_node) float64 dask.array<chunksize=(303714,), meta=np.ndarray>
    mhwtomsl     (N_node) float64 dask.array<chunksize=(303714,), meta=np.ndarray>
    mllwtomsl    (N_node) float64 dask.array<chunksize=(303714,), meta=np.ndarray>
    mlwtomsl     (N_node) float64 dask.array<chunksize=(303714,), meta=np.ndarray>
    navd88tomsl  (N_node) float64 dask.array<chunksize=(303714,), meta=np.ndarray>
Attributes:
    Conventions:  COARDS
    comment:      Vertical Datum transformations
    file_type:    Full_Grid
    grid_type:    unstructured
    history:      VDATUM Project
    institution:  NOAA/NOS/OCS/CSDL
    model:        Upgraded NGOFS
    references:   http://vdatum.noaa.gov
    source:       CSDL
    title:        Upgraded Northern Gulf of Mexico Operational Forecasting Sy...
    z_type:       sigma

In [8]:
# open dataset as zarr object using fsspec reference file system and xarray
ngofs_fs = fsspec.filesystem(
    "reference",
    fo="s3://nextgen-dmac-cloud-ingest/nos/ngofs2/nos.ngofs2.2ds.best.nc.zarr",
    target_protocol="s3",
    target_options={"anon": True},
    remote_protocol="s3",
    remote_options={"anon": True},
)
ngofs_mapper = ngofs_fs.get_mapper("")

ds_ngofs = xr.open_dataset(
    ngofs_mapper,
    engine="zarr",
    chunks={},
    backend_kwargs=dict(consolidated=False),
    drop_variables=["Itime", "Itime2"],
)
ds_ngofs

<xarray.Dataset>
Dimensions:             (time: 751, four: 4, nele: 569405, node: 303714,
                         three: 3, maxnode: 10, maxelem: 8)
Coordinates:
    lat                 (node) float32 dask.array<chunksize=(303714,), meta=np.ndarray>
    latc                (nele) float32 dask.array<chunksize=(569405,), meta=np.ndarray>
    lon                 (node) float32 dask.array<chunksize=(303714,), meta=np.ndarray>
    lonc                (nele) float32 dask.array<chunksize=(569405,), meta=np.ndarray>
  * time                (time) datetime64[ns] 2024-03-26T03:00:00 ... 2024-04...
Dimensions without coordinates: four, nele, node, three, maxnode, maxelem
Data variables: (12/38)
    Times               (time) |S26 dask.array<chunksize=(1,), meta=np.ndarray>
    a1u                 (four, nele) float32 dask.array<chunksize=(4, 569405), meta=np.ndarray>
    a2u                 (four, nele) float32 dask.array<chunksize=(4, 569405), meta=np.ndarray>
    art1                (node) float32 dask.array<chunksize=(303714,), meta=np.ndarray>
    art2                (node) float32 dask.array<chunksize=(303714,), meta=np.ndarray>
    atmos_press         (time, node) float32 dask.array<chunksize=(1, 303714), meta=np.ndarray>
    ...                  ...
    wet_nodes_prev_int  (time, node) int32 dask.array<chunksize=(1, 303714), meta=np.ndarray>
    x                   (node) float32 dask.array<chunksize=(303714,), meta=np.ndarray>
    xc                  (nele) float32 dask.array<chunksize=(569405,), meta=np.ndarray>
    y                   (node) float32 dask.array<chunksize=(303714,), meta=np.ndarray>
    yc                  (nele) float32 dask.array<chunksize=(569405,), meta=np.ndarray>
    zeta                (time, node) float32 dask.array<chunksize=(1, 303714), meta=np.ndarray>
Attributes: (12/14)
    Conventions:                 CF-1.0
    CoordinateProjection:        init=nad83:4205
    CoordinateSystem:            GeoReferenced
    GroundWater_Forcing:         GROUND WATER FORCING IS OFF!
    River_Forcing:               THERE ARE 63 RIVERS IN THIS MODEL.\nRIVER IN...
    Surface_Heat_Forcing:        FVCOM variable surface heat forcing file:\nF...
    ...                          ...
    Tidal_Forcing:               TIDAL ELEVATION FORCING IS OFF!
    history:                     model started at: 26/03/2024   03:15
    institution:                 School for Marine Science and Technology
    references:                  http://fvcom.smast.umassd.edu, http://codfis...
    source:                      FVCOM_4.3
    title:                       NGOFS2

In [36]:
datum_dims = [k for k in ds_vdatum.mllwtomsl.dims]
target_dims = [k for k in ds_ngofs.zeta.dims]
target_dims = target_dims[-len(datum_dims):]

new_dims = {o: k for (o, k) in zip(datum_dims, target_dims)}
# validate
for o, k in new_dims.items():
    assert ds_vdatum.mllwtomsl[o].shape == ds_ngofs.zeta[k].shape

ds_vdatum.mllwtomsl.rename(new_dims) + ds_ngofs.zeta

<xarray.DataArray (node: 303714, time: 751)>
dask.array<add, shape=(303714, 751), dtype=float64, chunksize=(303714, 1), chunktype=numpy.ndarray>
Coordinates:
    lat      (node) float32 dask.array<chunksize=(303714,), meta=np.ndarray>
    lon      (node) float32 dask.array<chunksize=(303714,), meta=np.ndarray>
  * time     (time) datetime64[ns] 2024-03-26T03:00:00 ... 2024-04-26T09:00:00
Dimensions without coordinates: node